In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp

In [0]:
def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

In [0]:
setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
rc = u.get_client("sge")

In [0]:
dview, lview = u.get_views(rc)
len(dview)

In [0]:
with dview.sync_imports():
    import os
    import sys
    import socket
    import stopwatch
    from subprocess import Popen, PIPE
    import tempfile
    import shutil

In [0]:
hlview = u.get_single_host_lview(rc, "all")
len(hlview)

In [0]:
bam_dir = "/home/cfriedline/eckertlab/gypsy_indiv/raw_demult_gbsx/analysis"
analysis_dir = os.path.join(bam_dir, "samtools1.3_masurca3")
if not os.path.exists(analysis_dir):
    os.makedirs(analysis_dir)
assert os.path.exists(analysis_dir)

In [0]:
bam_files = !find /home/cfriedline/eckertlab/gypsy_indiv/raw_demult_gbsx | grep masurca3 | grep 'rg.bam$' | grep -v OTIS
bam_files = [os.path.abspath(x) for x in bam_files if '.bam' in x]
len(bam_files)

In [0]:
samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"

In [0]:
assembly = "/home/cfriedline/eckertlab/projects/gypsy_moth/assemblies/masurca3/CA/10-gapclose/genome.ctg.fasta"

In [0]:
!$samtools faidx {assembly}

In [0]:
def call_snps(args):
    import socket, os, stopwatch
    print(socket.gethostname())
    timer = stopwatch.Timer()
    samtools, reference, bam_sorted, bcftools, raw_vcf, out_dir, contig_names = args 
    if not out_dir:
        out_dir = os.environ['TMPDIR']
    raw_vcf = os.path.join(out_dir, os.path.basename(raw_vcf))
    pileup = "%s mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR %s -Iugf %s %s | %s call -f GP,GQ -vmO z -o %s" % (samtools, 
                                                                     contig_names, reference, 
                                                                     ' '.join(bam_sorted), 
                                                                     bcftools,                                                                
                                                                     raw_vcf) 
    
    #print(pileup)
    #!$pileup
    timer.stop()
    return pileup, timer.elapsed

In [0]:
args = [samtools, 
        assembly, 
        bam_files, 
        bcftools, 
        "samtools_1.3.vcf.gz", 
        '/home/cfriedline/eckertlab/gypsy_indiv/raw_demult_gbsx/analysis/samtools1.3_masurca3/',
        ""]

In [0]:
pileup, elapsed = call_snps(args)

In [0]:
pileup

In [0]:
analysis_dir

In [0]:
with open(os.path.join(analysis_dir, "samtools.sh"), "w") as o:
    o.write("#!/bin/bash\n")
    o.write("%s\n" % pileup)

## Run on SGE
```bash
cd /gpfs_fs/home/eckertlab/gypsy_indiv/raw_demult_gbsx/analysis/samtools1.3_masurca3
qsub -cwd -N samtools -o samtools.out -e samtools.err -S /bin/bash -V -q all.q samtools.sh
```